In [1]:
import warnings
warnings.filterwarnings('ignore')

import os
import torch
from torch import nn

from hparams.hparams_GT import create_hparams
from datasets import LJDataset, TextMelCollate

In [2]:
!nvidia-smi
os.environ["CUDA_VISIBLE_DEVICES"]="0"

Tue Mar 23 20:14:31 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  Off  | 00000000:2E:00.0 Off |                  N/A |
| 53%   56C    P8    43W / 250W |   2660MiB / 11016MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  GeForce RTX 208...  Off  | 00000000:2F:00.0 Off |                  N/A |
| 26%   

In [3]:
from parallel_wavegan.utils import download_pretrained_model
from parallel_wavegan.utils import load_model

device='cuda'
vocoder_tag = "ljspeech_parallel_wavegan.v3"

# NOTE: Sometimes download is failed due to "Permission denied". That is 
#   the limitation of google drive. Please retry after serveral hours.
vocoder = load_model(download_pretrained_model(vocoder_tag)).to(device).eval()
vocoder.remove_weight_norm()
print('done')

from parallel_wavegan.utils import read_hdf5

stats_filename = "/home/rubato/.cache/parallel_wavegan/ljspeech_parallel_wavegan.v3/stats.h5"
mean = read_hdf5(stats_filename, 'mean')
scale = read_hdf5(stats_filename, 'scale')
print('done')

done
done


In [4]:
stt_hparams, tts_hparams = create_hparams()
testset = LJDataset(tts_hparams, split='test')
collate_fn = TextMelCollate(tts_hparams)

test_loader = torch.utils.data.DataLoader(testset, num_workers=1, 
                          shuffle=False, sampler=None, batch_size=1, pin_memory=False,
                          drop_last=True, collate_fn=collate_fn)
print(test_loader)

In [5]:
def to_cuda(batch):
    batch['text'] = batch['text'].cuda()
    batch['text_lengths'] = batch['text_lengths'].cuda()
    batch['mels'] = batch['mels'].cuda()
    batch['mel_lengths'] = batch['mel_lengths'].cuda()
    
    return batch

In [6]:
!mkdir inference/exp5/GTMEL

In [8]:
import matplotlib.pyplot as plt
import librosa.display
import numpy as np
import time 
import soundfile as sf

indexes = [15, 16, 26, 29, 31, 43, 44, 49, 51, 52, 53, 59, 70, 73, 75, 87, 94, 102, 116, 118, 121, 122, 124, 128, 133, 134, 142, 145, 146, 151, 154, 155, 165, 175, 176, 177, 183, 187, 189, 192, 193, 197, 204, 205, 213, 236, 245, 254, 256, 261, 274, 289, 293, 297, 299, 302, 303, 304, 305, 308, 309, 310, 312, 314, 316, 321, 323, 330, 331, 333, 338, 342, 346, 349, 350, 352, 364, 371, 373, 374, 382, 389, 391, 405, 416, 417, 421, 426, 428, 432, 439, 443, 445, 460, 474, 480, 481, 487, 489, 517]

inference_dir = 'inference/exp5/GTMEL/'
!mkdir $inference_dir

vocoder.eval()
with torch.no_grad():
    for i, batch in enumerate(test_loader):
        if i not in indexes[:50]:
            continue
        
        index = indexes.index(i)
        print(index, batch['strings'][0])
        batch = to_cuda(batch)
        _s = batch['mels'].data.cpu().numpy()
        _mel = _s[0].T
        _mel = (_mel - mean) / scale
        mel = torch.from_numpy(_mel).cuda()
        wav = vocoder.inference(mel)
        _wav = wav[:, 0].data.cpu().numpy()
        save_path = inference_dir + 'inference_' + str(index) + '_GTMEL.wav'
        sf.write(save_path, _wav, 22050, subtype='PCM_16')
        
print('done')

mkdir: `inference/exp5/GTMEL/' 디렉토리를 만들 수 없습니다: 파일이 있습니다
0 The Middle Ages brought calligraphy to perfection, and it was natural therefore
1 that the forms of printed letters should follow more or less closely those of the written character, and they followed them very closely.
2 especially as regards the lower-case letters; and type very similar was used during the next fifteen or twenty years not only by Schoeffer,
3 a very few years saw the birth of Roman character not only in Italy, but in Germany and France.
4 and used an exceedingly beautiful type, which is indeed to look at a transition between Gothic and Roman,
5 John of Spires and his brother Vindelin, followed by Nicholas Jenson, began to print in that city,
6 fourteen sixty-nine, fourteen seventy;
7 and though the famous family of Aldus restored its technical excellence, rejecting battered letters,
8 yet their type is artistically on a much lower level than Jenson's, and in fact
9 they must be considered to have ended the ag

In [13]:
import matplotlib.pyplot as plt
import librosa.display
import numpy as np
import time 
import soundfile as sf

indexes = [15, 16, 26, 29, 31, 43, 44, 49, 51, 52, 53, 59, 70, 73, 75, 87, 94, 102, 116, 118, 121, 122, 124, 126, 133, 134, 142, 145, 146, 151, 154, 155, 165, 175, 176, 177, 183, 187, 189, 192, 193, 197, 204, 205, 213, 236, 245, 254, 256, 261, 274, 289, 293, 297, 299, 302, 303, 304, 305, 308, 309, 310, 312, 314, 316, 321, 323, 330, 331, 333, 338, 342, 346, 349, 350, 352, 364, 371, 373, 374, 382, 389, 391, 405, 416, 417, 421, 426, 428, 432, 439, 443, 445, 460, 474, 480, 481, 487, 489, 517]

inference_dir = 'inference/exp3/GT/'
!mkdir $inference_dir

vocoder.eval()
with torch.no_grad():
    for i, batch in enumerate(test_loader):
        if i not in indexes[:50]:
            continue
        
        index = indexes.index(i)
        print(index, batch['strings'][0])
        batch = to_cuda(batch)
        _wav = batch['wavs'][0]
        save_path = inference_dir + 'inference_' + str(index) + '_GT.wav'
        sf.write(save_path, _wav, 22050, subtype='PCM_16')
        
print('done')

mkdir: `inference/exp3/GT/' 디렉토리를 만들 수 없습니다: 파일이 있습니다
0 The Middle Ages brought calligraphy to perfection, and it was natural therefore
1 that the forms of printed letters should follow more or less closely those of the written character, and they followed them very closely.
2 especially as regards the lower-case letters; and type very similar was used during the next fifteen or twenty years not only by Schoeffer,
3 a very few years saw the birth of Roman character not only in Italy, but in Germany and France.
4 and used an exceedingly beautiful type, which is indeed to look at a transition between Gothic and Roman,
5 John of Spires and his brother Vindelin, followed by Nicholas Jenson, began to print in that city,
6 fourteen sixty-nine, fourteen seventy;
7 and though the famous family of Aldus restored its technical excellence, rejecting battered letters,
8 yet their type is artistically on a much lower level than Jenson's, and in fact
9 they must be considered to have ended the age o